<a href="https://colab.research.google.com/github/ashishar/q_character/blob/main/Brahmi_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install pennylane torch torchvision torchaudio transformers pillow tqdm numpy
# ---- 1. Downgrade JAX to a PennyLane-compatible version ----
#!pip install -q "jax>=0.6.0,<0.6.3" "jaxlib>=0.6.0,<0.6.3" --force-reinstall
#
#!!pip install pennylane-lightning[gpu] cuquantum
#!pip install -q pennylane pennylane-lightning[gpu] cuquantum-python --extra-index-url=https://pypi.nvidia.com --no-cache-dir

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Change directory to the desired location where the file is stored.
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [4]:
# prompt: I want to download devanagari character dataset from Kaggle.

#!pip install kaggle
#!mkdir -p ~/.kaggle
#!cp /content/drive/MyDrive/brahmi/kaggle.json ~/.kaggle/kaggle.json
#!kaggle datasets download --force -d gautamneha/brahmi-dataset
!unzip "/content/drive/My Drive/brahmi-dataset.zip" -d "/content/data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/test/117/26.jpg  
  inflating: /content/data/test/117/27.jpg  
  inflating: /content/data/test/117/28.jpg  
  inflating: /content/data/test/117/29.jpg  
  inflating: /content/data/test/117/30.jpg  
  inflating: /content/data/test/118/26.jpg  
  inflating: /content/data/test/118/27.jpg  
  inflating: /content/data/test/118/28.jpg  
  inflating: /content/data/test/118/29.jpg  
  inflating: /content/data/test/118/30.jpg  
  inflating: /content/data/test/119/26.jpg  
  inflating: /content/data/test/119/27.jpg  
  inflating: /content/data/test/119/28.jpg  
  inflating: /content/data/test/119/29.jpg  
  inflating: /content/data/test/119/30.jpg  
  inflating: /content/data/test/12/1.jpg  
  inflating: /content/data/test/12/11.jpg  
  inflating: /content/data/test/12/15.jpg  
  inflating: /content/data/test/12/18.jpg  
  inflating: /content/data/test/12/22.jpg  
  inflating: /content/data/test/120/26.jpg  
  inflatin

In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, Subset

from torchvision import transforms
from skimage.morphology import skeletonize
from sklearn.preprocessing import binarize

from skimage import exposure
from skimage import io, transform
from skimage.transform import resize


def seam_carving(image, output_shape):
    """Apply seam carving to resize the image to the specified output shape."""
    image_np = image.numpy().squeeze()  # Convert to numpy and remove the channel dimension
    carved_image = resize(image_np, output_shape, anti_aliasing=True, mode='reflect')
    return torch.tensor(carved_image).unsqueeze(0)  # Convert back to tensor and add channel dimension


def binarize_image(image, threshold):
    return (image > threshold).float()

# Custom function to skeletonize the image
def skeletonize_image(image):
    image_np = image.numpy()  # Convert to NumPy array and remove channel dimension
    skeleton = skeletonize(image_np)  # Skeletonize the image
    return torch.tensor(skeleton).unsqueeze(0)  # Convert back to tensor and add channel dimension

# Custom function to flatten and normalize the image
def flatten_and_normalize(img):
    img = img.view(-1)  # Flatten the image
    img = img[:127]  # Use only the first 127 pixels
    #norm = img.norm(p=1)  # Calculate the L2 norm
    #img = img / norm  # Normalize the image
    return img

# Custom function for histogram normalization
def histogram_normalize(img):
    img_np = img.squeeze().numpy()  # Convert to NumPy array and remove channel dimension
    img_eq = exposure.equalize_hist(img_np)  # Apply histogram normalization
    return torch.tensor(img_eq).unsqueeze(0)  # Convert back to tensor and add channel dimension


image_size= 28
# Define the transform to resize, inspect size, flatten, and normalize the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((image_size, image_size)),  # Resize all images to 28x28 pixels
    transforms.ToTensor(),  # Convert to tensor
    transforms.Lambda(lambda x: seam_carving(x, (image_size, image_size))),  # Resize using seam carving
#    transforms.Lambda(lambda x: binarize_image(x, threshold=0.5)),  # Binarize the image
#    transforms.Lambda(skeletonize_image),  # Skeletonize the image
#    transforms.Lambda(lambda x: print(f"Image shape before flattening: {x.shape}") or x),  # Inspect size
    transforms.Lambda(flatten_and_normalize)  # Flatten and normalize
])

# Load the datasets with the defined transformation
train_dataset_all = datasets.ImageFolder(root='/content/data/train', transform=transform)
test_dataset_all = datasets.ImageFolder(root='/content/data/test', transform=transform)

batch_size=32
# Create DataLoaders
train_loader = DataLoader(train_dataset_all, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset_all, batch_size=batch_size, shuffle=False)

In [6]:
# --------------------------------------------------------------
#  1. Take only 1000 train + 300 test images
# --------------------------------------------------------------
from torch.utils.data import Subset, DataLoader
import random

# ---- reproducible random seed (optional) ----
random.seed(42)

# ---- training subset (1000 images) ----
train_indices = list(range(len(train_dataset_all)))
random.shuffle(train_indices)
train_indices = train_indices[:1000]                     # keep only first 1000
train_subset  = Subset(train_dataset_all, train_indices)

# ---- test subset (300 images) ----
test_indices = list(range(len(test_dataset_all)))
random.shuffle(test_indices)
test_indices = test_indices[:300]                        # keep only first 300
test_subset  = Subset(test_dataset_all, test_indices)

# ---- create the loaders (same batch_size you used before) ----
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_subset,  batch_size=batch_size, shuffle=False)

print(f"Train loader: {len(train_loader.dataset)} images")
print(f"Test  loader: {len(test_loader.dataset)} images")

Train loader: 1000 images
Test  loader: 300 images


In [ ]:
# --------------------------------------------------------------
#  Q-TrOCR 28×28 – FINAL ROBUST FIX (NO UNPACK ERROR)
# --------------------------------------------------------------

import os, random, warnings, torch, torch.nn as nn
import pennylane as qml
from pennylane import numpy as np
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms
from transformers import (
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from tqdm import tqdm

warnings.filterwarnings("ignore")

# ------------------- 0. CONFIG -------------------
IMAGE_SIZE   = 28
SEQ_LEN      = IMAGE_SIZE * IMAGE_SIZE
USE_FULL_SEQ = True
TRUNCATE_TO  = 127 if not USE_FULL_SEQ else SEQ_LEN
BATCH_SIZE   = 4
EPOCHS       = 10
LR           = 5e-5
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_DIR   = "./q_trocr_brahmi_gpu_final"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------- 1. SUBSET -------------------
random.seed(42)
train_idx = list(range(len(train_dataset_all)))[:100]
test_idx  = list(range(len(test_dataset_all)))[:30]
train_subset = Subset(train_dataset_all, train_idx)
test_subset  = Subset(test_dataset_all,  test_idx)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True,  pin_memory=False)
test_loader  = DataLoader(test_subset,  batch_size=BATCH_SIZE, shuffle=False, pin_memory=False)

print(f"Train: {len(train_subset)} | Test: {len(test_subset)}")

# ------------------- 2. TOKENIZER -------------------
def build_simple_tokenizer(dataset):
    classes = sorted(dataset.classes)
    special = ["<s>", "</s>", "<pad>", "<unk>"]
    vocab = {t:i for i,t in enumerate(special)}
    for i,c in enumerate(classes, start=len(special)): vocab[c] = i
    class SimpleTokenizer:
        def __init__(self): self.vocab = vocab; self.ids2tok = {v:k for k,v in vocab.items()}; self.vocab_size = len(vocab)
        def encode(self, txt): return [self.vocab.get(c, self.vocab["<unk>"]) for c in txt]
        def decode(self, ids, skip_special=True): return "".join(self.ids2tok.get(i,"<unk>") for i in ids if not (skip_special and i<4))
        def __call__(self, texts, **kwargs): return {"input_ids": [self.encode(t) for t in texts]}
    return SimpleTokenizer()

brahmi_tokenizer = build_simple_tokenizer(train_dataset_all)
print(f"Tokenizer: {brahmi_tokenizer.vocab_size} tokens | Using {TRUNCATE_TO} pixels")

# ------------------- 3. QUANTUM ENCODER (ROBUST) -------------------
dev = qml.device("lightning.gpu", wires=3)
EMBED_DIM = 8
pos_encoding = torch.nn.Parameter(torch.randn(TRUNCATE_TO, EMBED_DIM).to(DEVICE))
proj_weights = torch.nn.Parameter(torch.randn(EMBED_DIM, 3).to(DEVICE))

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def quantum_token_encoder(vec):
    qml.AmplitudeEmbedding(vec, wires=range(3), normalize=True)
    qml.Rot(*proj_weights[0], wires=0)
    qml.Rot(*proj_weights[1], wires=1)
    qml.Rot(*proj_weights[2], wires=2)
    return [qml.expval(qml.PauliZ(i)) for i in range(3)]

def quantum_sequence_encoder(pixel_values):
    batch_seq = pixel_values.squeeze(1)  # Remove channel dim
    # Remove any extra leading dims
    while batch_seq.dim() > 2:
        batch_seq = batch_seq.squeeze(0)
    # Ensure 2D: (B, L)
    if batch_seq.dim() == 1:
        batch_seq = batch_seq.unsqueeze(0)
    B, L = batch_seq.shape
    assert L == TRUNCATE_TO, f"Expected {TRUNCATE_TO}, got {L}"

    emb = batch_seq.unsqueeze(-1) * pos_encoding.unsqueeze(0)
    emb = emb / (emb.norm(dim=-1, keepdim=True) + 1e-8)
    emb_flat = emb.reshape(B * L, EMBED_DIM)
    states = torch.stack([quantum_token_encoder(emb_flat[i]) for i in range(B * L)])
    states = states.reshape(B, L, 3)
    return states.mean(dim=1)  # (B, 3)

# ------------------- 4. MODEL -------------------
class QTrOCR28x28(nn.Module):
    def __init__(self):
        super().__init__()
        dec = VisionEncoderDecoderModel.from_pretrained(
            "microsoft/trocr-base-handwritten",
            ignore_mismatched_sizes=True).decoder
        self.decoder = dec.to(DEVICE)
        self.decoder.config.vocab_size = brahmi_tokenizer.vocab_size
        self.proj = nn.Linear(3, 768).to(DEVICE)
        self.decoder.lm_head = nn.Linear(768, brahmi_tokenizer.vocab_size, bias=False).to(DEVICE)

    def forward(self, pixel_values, labels=None):
        pixel_values = pixel_values.to(DEVICE)
        labels = labels.to(DEVICE) if labels is not None else None
        q_out = quantum_sequence_encoder(pixel_values)
        hidden = self.proj(q_out).unsqueeze(1)
        return self.decoder(input_ids=labels, encoder_hidden_states=hidden, labels=labels)

model = QTrOCR28x28()

# ------------------- 5. LOADERWRAPPER -------------------
class LoaderWrapper:
    def __init__(self, loader, tokenizer):
        self.loader = loader
        self.tokenizer = tokenizer
        self.classes = loader.dataset.dataset.classes
        self.data = list(loader)
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        imgs, idxs = self.data[idx]
        pv = imgs.unsqueeze(1)  # (B, 1, TRUNCATE_TO)
        texts = [self.classes[i.item()] for i in idxs]
        ids = [self.tokenizer.encode(t) for t in texts]
        ids = [i + [1]*(10-len(i)) for i in ids]
        lbl = torch.tensor(ids, dtype=torch.long)
        return {"pixel_values": pv, "labels": lbl}

train_wrap = LoaderWrapper(train_loader, brahmi_tokenizer)
test_wrap  = LoaderWrapper(test_loader,  brahmi_tokenizer)

# ------------------- 6. TRAINING -------------------
args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    predict_with_generate=True,
    fp16=True,
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    report_to=[],
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_wrap,
    eval_dataset=test_wrap,
    tokenizer=brahmi_tokenizer,
)

print("=== TRAINING ON GPU (ROBUST SHAPE HANDLING) ===")
trainer.train()
trainer.save_model(OUTPUT_DIR)
torch.save({"pos":pos_encoding, "proj":proj_weights}, os.path.join(OUTPUT_DIR, "quantum.pth"))
print(f"Model saved to {OUTPUT_DIR}")

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


Train: 100 | Test: 30
Tokenizer: 174 tokens | Using 784 pixels


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

=== TRAINING ON GPU (ROBUST SHAPE HANDLING) ===
